In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [5]:
def call_open_meteo(selected_station):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": selected_station["lat"],
        "longitude": selected_station["lon"],
        "start_date": "1940-01-01",
        "end_date": "2024-02-29",
        "daily": "wind_speed_10m_max",
        "timezone": "Europe/Berlin",
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_wind_speed_10m_max = daily.Variables(0).ValuesAsNumpy()

    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left",
        ),
        "wind_speed_10m_max": daily_wind_speed_10m_max,
    }

    daily_dataframe = pd.DataFrame(data=daily_data)
    return daily_dataframe

In [6]:
zurich = {'lat':47.3744, 'lon':8.5411}

In [7]:
zurich_w = call_open_meteo(zurich)

zurich_w

,date,wind_speed_10m_max
0,1939-12-31 22:00:00+00:00,NaN
1,1940-01-01 22:00:00+00:00,12.496718
2,1940-01-02 22:00:00+00:00,6.608722
3,1940-01-03 22:00:00+00:00,7.289445
4,1940-01-04 22:00:00+00:00,6.109403
...,...,...
30736,2024-02-24 22:00:00+00:00,8.209263
30737,2024-02-25 22:00:00+00:00,7.280550
30738,2024-02-26 22:00:00+00:00,16.087610
30739,2024-02-27 22:00:00+00:00,12.261157
